In [1]:
# pip install pymorphy2 >> None

In [129]:
import pandas as pd
import numpy as np
import os
import re
import pymorphy2
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [3]:
# задаем директорию
file_path = 'C:/Users/user/Desktop/Hackathons/LinkedIn_menthoring/'

In [188]:
# открываем файлы с данными
posts = pd.read_csv(os.path.join(file_path, 'posts.csv'), index_col=0)
profiles = pd.read_csv(os.path.join(file_path, 'profiles.csv'), index_col=0)

In [189]:
# открываем датафрейм с постами
posts.head(2)

,user_id,text,likes,comments,reposts
0,ali-wodan,Кстати говоря. Теперь подкаст Миражи доступен в соцсети Вконтакте: https://lnkd.in/gKkrJX9Я наконец разобрался как туда прикрутить RSS :-) #podcast #миражи,1,0,0
1,ali-wodan,I’m #hiring. Know anyone who might be interested?,1,0,0


In [190]:
# открываем датафрейм с информацией о пользователях
profiles.head(2)

,id,user_name,user_head,user_work,user_position,user_tags,user_location,user_viewers,user_contacts,user_common_info
0,ali-wodan,Ali Wodan,Head of Design,Performix,Head Of Design,"podcast, it","Москва, Московская область, Россия",2 391,500+,"I am a digital product design lead. I've been designing digital systems for years. Teams creating, management, design process building, user research, analytics etc. it's all that i like to do. Also I am into psychology, music, philosophy."
1,ikotow,Игорь Котов,Директор по производству – Технократия,Технократия,Технократия,"it, обучение, менеджмент, технологии, производство","Казань, Республика Татарстан, Россия",340,338,Making IT production great again! Создатель сервиса для управления ресурсами: Resourcer.appМой telegram-канал: https://t.me/pastikotow


In [191]:
# переименуем столбец text в post для лучшего отражения содержимого
posts = posts.rename(columns={'text': 'post'})

**Предобработка данных**

In [192]:
# функция удаления эмодзи
def remove_emojis(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # смайлики
                               u"\U0001F300-\U0001F5FF"  # символы и пиктограммы
                               u"\U0001F680-\U0001F6FF"  # транспорт и символы на карте
                               u"\U0001F1E0-\U0001F1FF"  # флаги
                               u"\U00002500-\U00002BEF"  # китайские символы
                               # другие разные символы
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642" 
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    # Удаляем эмодзи, используя паттерны
    text_without_emojis = emoji_pattern.sub(r'', text)
    return text_without_emojis

# удаляем эмодзи из постов
posts['post'] = posts['post'].apply(lambda x: remove_emojis(x) if pd.notnull(x) else x)

In [193]:
# удалим посты на украинском языке

# определяем шаблон для украинских символов (по специфичным для данного языка символам)
ukrainian_pattern = r'[ЄєІіЇїҐґ]'

# создаем маску, указывающую строки, в которых столбец "post" содержит текст на украинском языке
mask = posts['post'].str.contains(ukrainian_pattern, regex=True, na=False)

# сохраняем в датафрейме только строки, в которых маска имеет значение False
posts = posts[~mask]

In [194]:
# сохраняем хэштэги в отдельный столбец перед их удалением из постов
posts['hashtags'] = posts['post'].str.findall(r'#([^\s]+)').apply(lambda x: ', '.join(x))

In [195]:
# функция лемматизации текста
morph = pymorphy2.MorphAnalyzer()
def lemmatize_text(text):
    lemmatized_words = [morph.parse(word)[0].normal_form for word in text.split()]
    return ' '.join(lemmatized_words)

# лемматизируем посты
posts['post_lemmatized'] = posts['post'].apply(lemmatize_text)

In [196]:
# удаляем слова, которые идут после хэш-тэга
posts['post_lemmatized'] = posts['post_lemmatized'].apply(lambda x: re.sub(r'#[^\s]+', ' ', x))

In [197]:
# производим замену дефиса на пробел
posts["post_lemmatized"] = posts["post_lemmatized"].str.replace("-", " ")

In [198]:
# удаляем лишние текстовые символы (те, которые не состоят из букв русского алфавита)
# только русские буквы и пробелы
posts['post_lemmatized'] = posts['post_lemmatized'].str.replace('[^а-яА-ЯёЁ\s]', ' ', regex=True) 

In [199]:
# скачиваем стоп-слова 
nltk.download('stopwords')
stop_words = set(stopwords.words('russian'))

# еще один список от bukvarix.com - список стоп-слов Яндекс Wordstat - (этот список можно дополнить/изменить)
file_path_words = os.path.join(file_path, 'stop_words.txt')
with open(file_path_words, 'r', encoding='utf-8') as file:
    stop_words_buk = file.read()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [200]:
# удаляем стоп-слова и слова-паразиты
posts['post_lemmatized'] = posts['post_lemmatized'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
posts['post_lemmatized'] = posts['post_lemmatized'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words_buk]))

In [201]:
'''если НЕ оставляем в постах английские слова'''
# определяем шаблон регулярного выражения для русских букв
pattern = '[^а-яА-ЯёЁ]'
# создаем маску, чтобы проверить, содержит ли каждая ячейка русские буквы
mask = posts['post_lemmatized'].str.contains(pattern, regex=True)
# фильтруем датафрейм, используя маску
posts = posts[mask]

In [202]:
posts.head()

,user_id,post,likes,comments,reposts,hashtags,post_lemmatized
0,ali-wodan,Кстати говоря. Теперь подкаст Миражи доступен в соцсети Вконтакте: https://lnkd.in/gKkrJX9Я наконец разобрался как туда прикрутить RSS :-) #podcast #миражи,1,0,0,"podcast, миражи",говоря подкаст мираж доступный соцсеть вконтакте разобраться прикрутить
2,ali-wodan,"Хэй честной народ! Ищу векторного иллюстратора на проект, с возможным длительным сотрудничеством по итогу. Можно удаленно. Уровень иллюстраций не хуже чем в примере https://lnkd.in/gkrvhxYРепост, пожалуйста)Если ты умеешь так рисовать, напиши мне в личку :-)Hey! I am looking for an Illustrator who can help us with a project! You need to have skills like in the example below or upper. Text me to discuss prices and due :-)#вакансия #vacancy #illustration #иллюстрация #project #проект",6,0,0,"вакансия, vacancy, illustration, иллюстрация, project, проект",честной народ искать векторный иллюстратор проект возможный длительный сотрудничество итогу удаленно уровень иллюстрация пример репост ести уметь рисовать написать личка
3,ali-wodan,"Новый пост подкаста ""Миражи"" на всех платформах:В аудио: https://lnkd.in/giWuSVNВ инстаграме: https://lnkd.in/gbV6yxKН #automotive # # #cars #а патреоне: https://lnkd.in/gfShqTU#саморазвитие #психология #психологиячеловека #психологияжизни #психологияличности #личностныйрост #подкаст #подкастмиражи #маркетинг #осознанность #мысли #мысливслух #установки #влияние #подсознаниеИспользованные звуки, музыка и картинки:Аудиоцитата из фильма ""Формула любви"" Марка ЗахароваJardins du Luxembourg by Jahzzar is licensed under a Attribution-ShareAlike 3.0 International License.prisoner by Luis Prado from the Noun Project",1,0,0,"automotive, cars, а, саморазвитие, психология, психологиячеловека, психологияжизни, психологияличности, личностныйрост, подкаст, подкастмиражи, маркетинг, осознанность, мысли, мысливслух, установки, влияние, подсознаниеИспользованные",пост подкаст миражи платформах аудио инстаграме патреоне звуки музыка картинки аудиоцитат фильм формула любви марк захарова
4,ali-wodan,"Подкаст Миражи, Эпизод 13Ошибка невозвратных затратpodcast.ru/1539345144#подкаст #podcast #it #podcasts #psychology #психология #miragespodcast",2,0,0,"подкаст, podcast, it, podcasts, psychology, психология, miragespodcast",подкаст миражи эпизод ошибка невозвратный затрат
5,ali-wodan,Новый эпизод об эффекте ИКЕА на всех платформах podcast.ru/1539345144,1,0,0,,эпизод эффект икеа платформа


In [203]:
posts.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2096 entries, 0 to 8597
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   user_id          2096 non-null   object
 1   post             2096 non-null   object
 2   likes            2096 non-null   object
 3   comments         2096 non-null   int64 
 4   reposts          2096 non-null   int64 
 5   hashtags         2096 non-null   object
 6   post_lemmatized  2096 non-null   object
dtypes: int64(2), object(5)
memory usage: 131.0+ KB


Объединяем датафреймы

In [204]:
# переименуем столбец id в user_id в датафрейме profiles, для последующего объединения с posts
profiles = profiles.rename(columns={'id': 'user_id'})

In [205]:
# объединяем датафреймы
df = pd.merge(posts, profiles, on='user_id')

In [206]:
# удаляем дубликаты
df.drop_duplicates(inplace=True)

In [207]:
# удаляем из столбца likes точки, запятые и пробелы
df["likes"] = df["likes"].replace(r'\.|\,|\s', '', regex=True)

# меняем тип данных сотлбца likes на integer
df["likes"] = df["likes"].astype("int64")

In [208]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2087 entries, 0 to 2095
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   user_id           2087 non-null   object
 1   post              2087 non-null   object
 2   likes             2087 non-null   int64 
 3   comments          2087 non-null   int64 
 4   reposts           2087 non-null   int64 
 5   hashtags          2087 non-null   object
 6   post_lemmatized   2087 non-null   object
 7   user_name         2087 non-null   object
 8   user_head         2087 non-null   object
 9   user_work         1999 non-null   object
 10  user_position     2087 non-null   object
 11  user_tags         397 non-null    object
 12  user_location     2080 non-null   object
 13  user_viewers      2087 non-null   object
 14  user_contacts     2087 non-null   object
 15  user_common_info  1898 non-null   object
dtypes: int64(3), object(13)
memory usage: 277.2+ KB


In [209]:
# Сохраняем датафрейм
df.to_csv(os.path.join(file_path, 'linkedin.csv'))

# Сохраняем датафрейм лемматизации
df[[
    'user_id', 'post_lemmatized', 'likes', 'comments', 'reposts'
]].to_csv(os.path.join(file_path, 'post_lemmatized.csv'))

In [210]:
df.sample(5)

,user_id,post,likes,comments,reposts,hashtags,post_lemmatized,user_name,user_head,user_work,user_position,user_tags,user_location,user_viewers,user_contacts,user_common_info
1077,surenharutyunyanc,"Ознакомиться с сайтом likesuro.com. Просто наберите его название в поисковике и перейдите на страницу ""FOR CANDIDATES"" или ""КАНДИДАТАМ"", если вы на русскоязычной странице. #поискработы #резюме #анкета #likesuro #CRM #рекрутер #вакансии #кандидаты #квалификация #шансы #карьера #работа #собеседование",0,0,0,"поискработы, резюме, анкета, likesuro, CRM, рекрутер, вакансии, кандидаты, квалификация, шансы, карьера, работа, собеседование",ознакомиться сайт набрать название поисковик перейти страница кандидатам русскоязычный странице,Suren Harutyunyan,Recruter,Университет имени Жиули Шартавы,"IT recruiter, sourcer",NaN,"Tbilisi, Georgia","2,067",500+,"A brief message about myself as a recruiter could look like this:I am a professional recruiter with experience in the IT industry. I specialize in sourcing and recruiting talented professionals and possess strong sourcing skills, as well as the ability to work with candidates at all stages of the hiring process. I strive to create a positive atmosphere and aim to find the ideal candidate for each vacancy."
90,agratoth,"Весьма интересная и нестандартная вакансия! Highload-стек, возможность повысить свою экспертизу и делиться ей с другими#vacancy #highload #clickhouse #hadoop",1,0,0,"vacancy, highload, clickhouse, hadoop",весьма интересный нестандартный вакансия стек возможность повысить экспертиза делиться другими,Anton Berdnikov,CTO/Co-founder,Cloudberry.bi,Chief Technical Officer/Co-founder,NaN,Russia,"4,569",500+,Разработка нейросетей на python/keras.io + tensorflowРазработка микросервисов на golangРазработка высоконагруженных приложенийСистемная разработка на golangСистемная разработка на C/C++
574,drazd,"Еще одна победа над рутиной - на этот раз в поле рекрутинга. Особенно сегодня, когда в пост-ковидную эпоху работу через Интернет научились искать практически все - это решение очень к месту! Знаю, что среди моих подписчиков и друзей есть рекрутёры, присмотритесь повнимательнее!",3,0,0,,победа рутина поле рекрутинга особенно пост ковидный эпоха работа интернет научиться искать практически решение месту знаю подписчик рекрутёры присмотреться повнимательнее,Valentin Drazdov,"Product Manager in PIX Robotics | IT Expert. Trusted advisor in RPA, ECM, BPM in Russia & CIS",PIX Robotics,Product Manager,NaN,"Moscow, Moscow City, Russia",634,500+,"The most important thing in IT - is to understand customer's pain, translate it to developers team and provide the very best solution for end users. All of my 10+ years career I parcitipated in many big projects as: developer, lead, architect, pre-sales and even project manager. My experience giving great opportunity to make projects easier to achieve the goal at any stage. If you had some problems with big IT systems in your company, or even you had bad experience with some ECM&BPM solutions - let's discuss about that and maybe I will show you the way to better feature of IT systems in your company."
1300,linkwolf,Друзья! Директора и топ-менеджеры компаний. Моя компания Digital Agency Linkwolf предлагает вам услуги по созданию сайта и бренда. Если вам интересно вы можете написать мне либо оставить комментарий.www.linkwolf.netЗа репост +100 к карма #разработка,3,0,1,разработка,друзья директор топ менеджер компаний компания предлагать услуга создание сайт бренда написать либо оставить комментарий репост карма,Nikita Tulubaiev,I'm Nikita. Hello everyone!,"University of Economics and Law ""KROK""",Middle full stack,NaN,"Kyiv, Kyiv City, Ukraine","2,856",500+,I'm Nikita. Hello everyone!
1404,elizaveta-kalinina-3aab58214,А вы уже опробовали тренажёр по математике от Yandex?Неплохой способ оценить свои знания в математике),5,0,1,,опробовать тренажёр математика неплохой способ оценить знание математике,Elizaveta Kalinina,UX designer / Product designer,Internat

---

In [211]:
docs = df['post_lemmatized'].tolist()

In [212]:
tfidf = TfidfVectorizer(min_df=10, max_df=0.9)

In [213]:
x = tfidf.fit_transform(docs)

In [214]:
tfidf.fit(docs)

TfidfVectorizer(max_df=0.9, min_df=10)

In [215]:
x.shape

(2087, 1634)

In [216]:
id2word = {i: token for token, i in tfidf.vocabulary_.items()}

**LDA**

In [217]:
n_topics = 20

In [218]:
lda = LatentDirichletAllocation(n_components=n_topics, random_state=31)

In [219]:
topics = lda.fit_transform(x)

In [220]:
topics.shape

(2087, 20)

**Ключевые слова**

In [221]:
for i in range(n_topics):
    c = lda.components_[i, :]
    topic_tokens = [id2word.get(token_id) for token_id in np.argsort(c)[-20:]]
    print("Тема",i+1, ":")
    print(", ".join(topic_tokens))
    print("\n ")

Тема 1 :
анимация, полезный, выполнить, резюме, полезно, ссылка, считать, медицинский, проект, поговорить, долгий, коммерческий, москве, защита, коллеги, советовать, сообщить, специалистов, сеть, находка

 
Тема 2 :
выступать, давно, нанимать, выстраивать, рассказать, карта, секрет, проекта, дальше, проект, скидка, отрасли, статья, тематика, бонус, итог, команда, поддержать, эксперт, спикер

 
Тема 3 :
млн, делиться, рисовать, активно, просмотреть, специалист, сайт, продукт, страница, работу, проект, директ, рф, статью, искать, персонал, наставник, услуг, посетить, банк

 
Тема 4 :
приглашать, показать, проходить, открытие, конференции, направление, яндекс, поучаствовать, компания, школа, интенсив, сменить, курс, красиво, удовольствие, скидка, получил, онлайн, ловить, сертификат

 
Тема 5 :
снижение, оптимизировать, канал, вебинар, таймкоды, бизнес, создать, прибыль, рассказывать, инвестиция, картинка, расчёт, полный, дополнительный, материал, формула, рабочий, схема, видео, тетрадь

 

**Типичные статьи**

In [222]:
for i in range(n_topics):
    doc_id = np.argmax(topics[:, i])
    print("Тема ", i)
    print(df.iloc[doc_id]["post"])
    print("\n")

Тема  0
We have developed the structure and design of the website. Then we performed adaptive layout on Tilda.A mail server has been set up to receive applications. The certificates are also watermarked for copy protection.Prohect's link https://lnkd.in/d3sMJ7cX#design #website #abinur =========================Мы разработали структуру и дизайн сайта. Затем мы выполнили адаптивную верстку на Тильде.Настроен почтовый сервер для приема заявок. Сертификаты также снабжены водяными знаками для защиты от копирования.ссылка на проект https://lnkd.in/d3sMJ7cX


Тема  1
New post about Security Awareness in Russia Security Awareness. Повышаем осведомленность в области ИБПоговорили с экспертами ИБ-отрасли о Security Awareness. Расспросили их об актуальных киберугрозах, о пресловутом «человеческом факторе», о росте спроса на услуги по повышению осведомленности по вопросам ИБ, об особенностях применения SA на практике.https://lnkd.in/eFr_JN2g #security #кибербезопасность #иб #SecurityAwareness #info